In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요한 module import 및 trainset, testset 만들기

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torchvision.models as models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

trainset = dset.CIFAR10(root="../", train = True, transform = transforms.ToTensor(), download = True)
testset = dset.CIFAR10(root="../", train = False, transform = transforms.ToTensor(), download = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../cifar-10-python.tar.gz to ../
Files already downloaded and verified


Problem 1 - 시도 1. 사전훈련된 VGG 학습하기

*   learning rate = 0.0001
*   batch size = 32

In [ ]:
# 수업시간에 사용하였던 모델, learning rate, batch size라 그대로 선택함.

batch_size_1 = 32
learning_rate_1 = 0.0001
num_epoch = 50

train_loader_1 = DataLoader(trainset, batch_size = batch_size_1)
test_loader_1 = DataLoader(testset, batch_size = batch_size_1)

model1 = models.vgg16(pretrained = True)
model1.classifier[6] = nn.Linear(in_features = 4096, out_features = 10) # CIFAR10 dataset에 맞게 모델 구조 변경
model1.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr = learning_rate_1)

for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader_1):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model1.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(0.9074, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3036, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0043, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0979, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0026, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model1.eval()
    for data in test_loader_1:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model1(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("model 1 accuacy : ", 100 * correct / total)

model 1 accuacy :  87.64


In [ ]:
torch.save(model1, '/content/drive/MyDrive/Colab Notebooks/final_project/model1.pt')

Problem 1 - 시도 2. 사전훈련된 VGG 학습하기
*   learning rate = 0.00001
*   batch size = 32

In [ ]:
# 같은 VGG에서 learning rate를 줄였을때 성능의 변화가 궁금함. 

batch_size_2 = 32
learning_rate_2 = 0.00001
num_epoch = 50

train_loader_2 = DataLoader(trainset, batch_size = batch_size_2)
test_loader_2 = DataLoader(testset, batch_size = batch_size_2)

model2 = models.vgg16(pretrained = True)
model2.classifier[6] = nn.Linear(in_features = 4096, out_features = 10) # CIFAR10 dataset에 맞게 모델 구조 변경
model2.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr = learning_rate_2)

for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader_2):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model2.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(1.0755, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0081, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0022, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0028, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0131, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model2.eval()
    for data in test_loader_2:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("model 2 accuacy : ", 100 * correct / total)

model 2 accuacy :  86.56


In [ ]:
torch.save(model2, '/content/drive/MyDrive/Colab Notebooks/final_project/model2.pt')

Problem 1 - 시도 3. 사전훈련된 VGG 학습하기
*   learning rate = 0.00001
*   batch size = 128

In [ ]:
# 같은 VGG 모델에서 batch size를 늘렸을 때의 성능 변화가 궁금함.

model3 = models.vgg16(pretrained = True)
model3.classifier[6] = nn.Linear(in_features = 4096, out_features = 10)
model3.to(device)

batch_size_3 = 128
learning_rate_3 = 0.0001
num_epoch = 50

train_loader_3 = DataLoader(trainset, batch_size = batch_size_3)
test_loader_3 = DataLoader(testset, batch_size = batch_size_3)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr = learning_rate_3)

for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader_3):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model3.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(0.5790, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0167, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0012, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0011, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0008, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model3.eval()
    for data in test_loader_3:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model3(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("model 3 accuacy : ", 100 * correct / total)

model 3 accuacy :  88.54


In [ ]:
torch.save(model3, '/content/drive/MyDrive/Colab Notebooks/final_project/model3.pt')

problem 1 - 시도 4. 사전훈련된 VGG 학습하기
*   learning rate = 0.0001
*   batch size = 256

In [ ]:
# batch size를 늘렸을 때 성능이 향상되는 것 같아 시도 3보다 더 늘려봄.

model4 = models.vgg16(pretrained = True)
model4.classifier[6] = nn.Linear(in_features = 4096, out_features = 10)
model4.to(device)

batch_size_4 = 256
learning_rate_4 = 0.0001
num_epoch = 50

train_loader_4 = DataLoader(trainset, batch_size = batch_size_4)
test_loader_4 = DataLoader(testset, batch_size = batch_size_4)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model4.parameters(), lr = learning_rate_4)

for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader_4):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model4.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(0.6021, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0009, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0033, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.6441e-05, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model4.eval()
    for data in test_loader_4:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model4(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("model 4 accuacy : ", 100 * correct / total)

model 4 accuacy :  87.67


In [ ]:
torch.save(model4, '/content/drive/MyDrive/Colab Notebooks/final_project/model4.pt')


# Problem 1 결과
최종 성능 88.54%로 model3 (사전 훈련된 VGG16을 learning rate = 0.0001, batch size = 128으로 학습시키기)에서 가장 높은 성능을 기록함.